In [3]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\api_scraping")
    sys.path.append(module_path+"\\functions")

from get_user_data import *
from database import *

In [4]:
print(getUserPlaylists())

{'irish tavern': '4JGY9SfDvvTGecTSaTGFUV', 'country?': '5J0lP9PmQb4XBebDt1O5KE', 'gym electropop': '6T18XkG7ZbovwypPQFTm55', 'gym metal-rock': '0uDBW7RKRAl81WDF7iNBgw', 'powerwolf': '0DqzAhHJN82fZhSFFawdUH', 'musica papa': '4p16STMl2UrZ368yMV3cgE', 'Dean Lewis: Complete Collection': '1yTgfZuOGyKbpvh8oNKoUW', 'piano': '0aBenURYxCeUoTBOc94EJm', 'miau': '0K0kKdTp7DonYfU3JZbqSX', "chillin'": '2ivUxiPzbA4Uer5yrkxi2W', "iiii]; )'": '3uHu9xWtS4AI8T9Br5VKk3', 'power': '4TUeAPGhUFS9TAUBWRauAK', 'jordii': '7hOk6IREOVrvVAvyPZoPUs'}
